## response collection

The purpose of this notebook is to compare the model predictions saved to a mongodb
collection with other sources of information about the site in the model at the time
of the astronomical twilight in the response.


In [76]:
import typing as t
from dataclasses import dataclass
from pathlib import Path
from datetime import datetime, timedelta

import httpx
from pymongo import MongoClient

DB_NAME = "validation_data"
COLLECTION_NAME = "api_response"

coordinates = (-30.2466, -70.7494)
lat, lon = coordinates


@dataclass
class Site:
    lat: str
    lon: str
    astro_twilight_datetime: datetime


with open(Path.cwd() / ".env") as f:
    first_line, *_ = f.read().split("\n")
    idx = first_line.find("=")
    uri = first_line[idx + 1 :]


def get_db():
    client = MongoClient(uri)
    return client[DB_NAME]


db = get_db()
db

Database(MongoClient(host=['ac-atnkhx8-shard-00-00.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-02.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-01.7a9iiun.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, ssl=True, ssl_cert_reqs=<VerifyMode.CERT_NONE: 0>, authsource='admin', replicaset='atlas-3ats48-shard-0'), 'validation_data')

In [77]:
collection = db[COLLECTION_NAME]
collection

Collection(Database(MongoClient(host=['ac-atnkhx8-shard-00-00.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-02.7a9iiun.mongodb.net:27017', 'ac-atnkhx8-shard-00-01.7a9iiun.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, ssl=True, ssl_cert_reqs=<VerifyMode.CERT_NONE: 0>, authsource='admin', replicaset='atlas-3ats48-shard-0'), 'validation_data'), 'api_response')

In [78]:
async def get_prediction(coordinates: t.Tuple):
    async with httpx.AsyncClient() as client:
        url = f"http://localhost:8000/api/prediction?lat={lat}&lon={lon}"
        r = await client.get(url)
        return r.json()


async def insert_prediction():
    prediction = await get_prediction(coordinates)
    collection.insert_one(prediction)
    return prediction


prediction = await insert_prediction()
prediction

{'brightness_mpsas': 19.7863,
 'astro_twilight': {'iso': '2024-01-01 01:24:20.546', 'type': 'nearest'},
 '_id': ObjectId('6591e6ebb29b8604829db6cd')}

In [79]:
open_meteo_url = "https://archive-api.open-meteo.com/v1"


async def get_historical_meteo_data_at_site(site: Site) -> t.Any:
    def format_datetime(dt: datetime):
        return dt.strftime("%Y-%m-%d")

    astro_twilight_plus_one = site.astro_twilight_datetime + timedelta(days=0)
    start_date_str = format_datetime(site.astro_twilight_datetime)
    end_date_str = format_datetime(astro_twilight_plus_one)
    url = f"{open_meteo_url}/archive?latitude={site.lat}&longitude={site.lon}&start_date={start_date_str}&end_date={end_date_str}&hourly=temperature_2m,cloud_cover"
    print(url)
    async with httpx.AsyncClient() as client:
        r = await client.get(url)
        r.raise_for_status()
        json = r.json()
        return json["hourly"]["cloud_cover"][site.astro_twilight_datetime.hour]

In [80]:
items_in_collection = list(collection.find())
items_in_collection

[{'_id': ObjectId('6590b4a4b29b8604829db6ba'),
  'brightness_mpsas': 19.7034,
  'astro_twilight': {'iso': '2023-12-31 01:24:12.658 UTC', 'type': 'nearest'}},
 {'_id': ObjectId('6590b4ccb29b8604829db6bb'),
  'brightness_mpsas': 19.7034,
  'astro_twilight': {'iso': '2023-12-31 01:24:12.323 UTC', 'type': 'nearest'}},
 {'_id': ObjectId('6590b6ceb29b8604829db6bd'),
  'brightness_mpsas': 19.7033,
  'astro_twilight': {'iso': '2023-12-31 01:24:12.828 UTC', 'type': 'nearest'}},
 {'_id': ObjectId('6590b6fcb29b8604829db6bf'),
  'brightness_mpsas': 19.7034,
  'astro_twilight': {'iso': '2023-12-31 01:24:12.503 UTC', 'type': 'nearest'}},
 {'_id': ObjectId('6590ba81b29b8604829db6c0'),
  'brightness_mpsas': 19.7034,
  'astro_twilight': {'iso': '2023-12-31 01:24:13.074 UTC', 'type': 'nearest'}},
 {'_id': ObjectId('6590c0d4b29b8604829db6c4'),
  'brightness_mpsas': 19.7034,
  'astro_twilight': {'iso': '2023-12-31 01:24:12.324 UTC', 'type': 'nearest'}},
 {'_id': ObjectId('6590c67ab29b8604829db6c6'),
  'br

In [81]:
for item in items_in_collection:
    astro_twilight_iso = item["astro_twilight"]["iso"]
    astro_twilight_datetime = datetime.strptime(
        astro_twilight_iso, "%Y-%m-%d %H:%M:%S.%f"
    )
    s = Site(
        lat=lat,
        lon=lon,
        astro_twilight_datetime=astro_twilight_datetime,
    )
    meteo_data = await get_historical_meteo_data_at_site(s)
    print(meteo_data)

ValueError: unconverted data remains:  UTC